In [7]:
# ! ln ../bcc/inputs/163_A1*.npy inputs
# ! ln ../bcc/gnn_models/4.model.pth models/tumor_map_gnn.pth
# ! ln ../bcc/pretrain_model.pth models/tumor_map_cnn.pth
# ! ln ../fat_dermis_epi_sq_model/v2/checkpoints_tissue_seg/104.checkpoint.pth models/macro_map_cnn.pth
# ! ln ../nuclei_pipeline/seg_model/27.checkpoint.pth models/nuclei.pth
# torch.save(GCNNet(2048, 4, [32]*3).state_dict(),"models/macro_map_gnn.pth")
# ! pip uninstall pathpretrain -y && pip install git+https://github.com/jlevy44/PathPretrain
! cd ../ArcticAI_Prototype/ && git add * */* && git commit -a -m "workflow update" && git push

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [ ]:
# preprocess

In [24]:
import os, tqdm
import numpy as np, pandas as pd
from pathflowai.utils import generate_tissue_mask
from itertools import product
from scipy.ndimage.morphology import binary_fill_holes as fill_holes

def preprocess(basename="163_A1a",
               threshold=0.05,
               patch_size=256):
    
    image=f"inputs/{basename}.npy"
    basename=os.path.basename(image).replace('.npy','')
    image=np.load(image)
    
    masks=dict()
    masks['tumor_map']=generate_tissue_mask(image,
                             compression=10,
                             otsu=False,
                             threshold=240,
                             connectivity=8,
                             kernel=5,
                             min_object_size=100000,
                             return_convex_hull=False,
                             keep_holes=False,
                             max_hole_size=6000,
                             gray_before_close=True,
                             blur_size=51) 
    x_max,y_max=masks['tumor_map'].shape
    masks['macro_map']=fill_holes(masks['tumor_map'])
    
    patch_info=dict()
    for k in masks:
        patch_info[k]=pd.DataFrame([[basename,x,y,patch_size,"0"] for x,y in tqdm.tqdm(list(product(range(0,x_max-patch_size,patch_size),range(0,y_max-patch_size,patch_size))))],columns=['ID','x','y','patch_size','annotation'])
        patches=np.stack([image[x:x+patch_size,y:y+patch_size] for x,y in tqdm.tqdm(patch_info[k][['x','y']].values.tolist())])                   
        include_patches=np.stack([masks[k][x:x+patch_size,y:y+patch_size] for x,y in tqdm.tqdm(patch_info[k][['x','y']].values.tolist())]).mean((1,2))>=threshold

        np.save(f"masks/{basename}_{k}.npy",masks[k])
        np.save(f"patches/{basename}_{k}.npy",patches[include_patches]) 
        patch_info[k].iloc[include_patches].to_pickle(f"patches/{basename}_{k}.pkl")



In [2]:
# preprocess(basename="163_A1a",
#                threshold=0.05,
#                patch_size=256)

In [3]:
# test component

In [25]:
# predict model
import os, torch, tqdm, pandas as pd, numpy as np
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from pathpretrain.train_model import train_model, generate_transformers, generate_kornia_transforms

class CustomDataset(Dataset):
    # load using saved patches and mask file
    def __init__(self, patch_info, npy_file, transform):
        self.X=np.load(npy_file)
        self.patch_info=pd.read_pickle(patch_info)
        self.xy=self.patch_info[['x','y']].values
        self.patch_size=self.patch_info['patch_size'].iloc[0]
        self.length=self.patch_info.shape[0]
        self.transform=transform
        self.to_pil=lambda x: Image.fromarray(x)
        self.ID=os.path.basename(npy_file).replace(".npy","")
        
    def __getitem__(self,i):
        x,y=self.xy[i]
        return self.transform(self.to_pil(self.X[i]))#[x:x+patch_size,y:y+patch_size]
        
    def __len__(self):
        return self.length
    
    def embed(self,model,batch_size,out_dir):
        Z=[]
        dataloader=DataLoader(self,batch_size=batch_size,shuffle=False)
        n_batches=len(self)//batch_size
        with torch.no_grad():
            for i,X in tqdm.tqdm(enumerate(dataloader),total=n_batches):
                if torch.cuda.is_available(): X=X.cuda()
                z=model(X).detach().cpu().numpy()
                Z.append(z)
        Z=np.vstack(Z)
        torch.save(dict(embeddings=Z,patch_info=self.patch_info),os.path.join(out_dir,f"{self.ID}.pkl"))
        
def generate_embeddings(basename="163_A1a",
                        analysis_type="tumor",
                       gpu_id=0):
    patch_info_file,npy_file=f"patches/{basename}_{analysis_type}_map.pkl",f"patches/{basename}_{analysis_type}_map.npy"
    models={k:f"models/{k}_map_cnn.pth" for k in ['macro','tumor']}
    num_classes=dict(macro=4,tumor=2)
    train_model(model_save_loc=models[analysis_type],extract_embeddings=True,num_classes=num_classes[analysis_type],predict=True,embedding_out_dir="cnn_embeddings/",custom_dataset=CustomDataset(patch_info_file,npy_file,generate_transformers(224,256)['test']),gpu_id=gpu_id)



In [5]:
# generate_embeddings(basename="163_A1a",
#                         analysis_type="tumor",
#                        gpu_id=0)
# generate_embeddings(basename="163_A1a",
#                         analysis_type="macro",
#                        gpu_id=0)

In [6]:
# create graph

In [7]:
import os, torch, numpy as np, pandas as pd
import pickle
import scipy.sparse as sps
from torch_geometric.utils import subgraph, add_remaining_self_loops
from torch_cluster import radius_graph
from collections import Counter
from torch_geometric.data import Data 

def create_graph_data(basename="163_A1a",
                      analysis_type="tumor",
                      radius=256,
                      min_component_size=600):
    embeddings=torch.load(f"cnn_embeddings/{basename}_{analysis_type}_map.pkl")
    xy=torch.tensor(embeddings['patch_info'][['x','y']].values).float().cuda()
    X=torch.tensor(embeddings['embeddings'])
    G=radius_graph(xy, r=radius*np.sqrt(2), batch=None, loop=True)
    G=G.detach().cpu()
    G=add_remaining_self_loops(G)[0]
    xy=xy.detach().cpu()
    datasets=[]
    edges=G.detach().cpu().numpy().astype(int)
    n_components,components=list(sps.csgraph.connected_components(sps.coo_matrix((np.ones_like(edges[0]),(edges[0],edges[1])))))
    comp_count=Counter(components)
    components=torch.LongTensor(components)
    for i in range(n_components):
        if comp_count[i]>=min_component_size:
            G_new=subgraph(components==i,G,relabel_nodes=True)[0]
            xy_new=xy[components==i]
            X_new=X[components==i]
            np.random.seed(42)
            idx=np.arange(X_new.shape[0])
            idx2=np.arange(X_new.shape[0])
            np.random.shuffle(idx)
            train_idx,val_idx,test_idx=torch.tensor(np.isin(idx2,idx[:int(0.8*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.8*len(idx)):int(0.9*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.9*len(idx)):]))
            dataset=Data(x=X_new, edge_index=G_new, y_new=torch.ones(len(X_new)), edge_attr=None, pos=xy_new)
            dataset.train_mask=train_idx
            dataset.val_mask=val_idx
            dataset.test_mask=test_idx
            dataset.id=basename
            dataset.component=i
            datasets.append(dataset)
    pickle.dump(datasets,open(os.path.join('graph_datasets',f"{basename}_{analysis_type}_map.pkl"),'wb'))

In [8]:
# create_graph_data(basename="163_A1a",
#                       analysis_type="tumor",
#                       radius=256,
#                       min_component_size=600)
# create_graph_data(basename="163_A1a",
#                       analysis_type="macro",
#                       radius=256,
#                       min_component_size=600)

In [9]:
# predict graph

In [29]:
import os, torch, pickle, numpy as np, pandas as pd, torch.nn as nn
from torch_geometric.data import DataLoader as TG_DataLoader
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse, dropout_adj, to_networkx
from torch_geometric.nn import GATConv
import torch.nn.functional as F

class GCNNet(torch.nn.Module):
    def __init__(self, inp_dim, out_dim, hidden_topology=[32,64,128,128], p=0.5, p2=0.1, drop_each=True):
        super(GCNNet, self).__init__()
        self.out_dim=out_dim
        self.convs = nn.ModuleList([GATConv(inp_dim, hidden_topology[0])]+[GATConv(hidden_topology[i],hidden_topology[i+1]) for i in range(len(hidden_topology[:-1]))])
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.dropout = nn.Dropout(p)
        self.fc = nn.Linear(hidden_topology[-1], out_dim)
        self.drop_each=drop_each

    def forward(self, x, edge_index, edge_attr=None):
        for conv in self.convs:
            if self.drop_each and self.training: edge_index=self.drop_edge(edge_index)
            x = F.relu(conv(x, edge_index, edge_attr))
        if self.training:
            x = self.dropout(x)
        x = self.fc(x)
        return x
    
class GCNFeatures(torch.nn.Module):
    def __init__(self, gcn, bayes=False, p=0.05, p2=0.1):
        super(GCNFeatures, self).__init__()
        self.gcn=gcn
        self.drop_each=bayes
        self.gcn.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.gcn.dropout = nn.Dropout(p)
    
    def forward(self, x, edge_index, edge_attr=None):
        for i,conv in enumerate(self.gcn.convs):
            if self.drop_each: edge_index=self.gcn.drop_edge(edge_index)
            x = conv(x, edge_index, edge_attr)
            if i+1<len(self.gcn.convs):
                x=F.relu(x)
        if self.drop_each:
            x = self.gcn.dropout(x)
        y = self.gcn.fc(F.relu(x))#F.softmax()
        return x,y
    
def predict(basename="163_A1a",
            analysis_type="tumor",
            gpu_id=0):
    hidden_topology=dict(tumor=[32]*3,macro=[32]*3)
    num_classes=dict(macro=4,tumor=2)
    torch.cuda.set_device(gpu_id)
    dataset=pickle.load(open(os.path.join('graph_datasets',f"{basename}_{analysis_type}_map.pkl"),'rb'))
    model=GCNNet(dataset[0].x.shape[1],num_classes[analysis_type],hidden_topology=hidden_topology[analysis_type],p=0.,p2=0.)
    model=model.cuda()
    model.load_state_dict(torch.load(os.path.join("models",f"{analysis_type}_map_gnn.pth"),map_location=f"cuda:{gpu_id}" if gpu_id>=0 else "cpu"))
    dataloader=TG_DataLoader(dataset,shuffle=False,batch_size=1)
    model.eval()
    feature_extractor=GCNFeatures(model,bayes=False).cuda()
    graphs=[]
    for i,data in enumerate(dataloader):
        with torch.no_grad():
            graph = to_networkx(data).to_undirected()
            model.train(False)
            x=data.x.cuda()
            xy=data.pos.numpy()
            edge_index=data.edge_index.cuda()
            preds=feature_extractor(x,edge_index)
            z,y_pred=preds[0].detach().cpu().numpy(),preds[1].detach().cpu().numpy()
            graphs.append(dict(G=graph,xy=xy,z=z,y_pred=y_pred,slide=data.id,component=data.component))
    torch.save(graphs,os.path.join("gnn_results",f"{basename}_{analysis_type}_map.pkl"))


In [30]:
# predict(basename="163_A1a",
#             analysis_type="tumor",
#             gpu_id=0)
# predict(basename="163_A1a",
#             analysis_type="macro",
#             gpu_id=0)

In [31]:
# nuclei prediction

In [40]:
from PIL import Image
from torch.utils.data import Dataset
import torch, pandas as pd, numpy as np
import pickle
from pathpretrain.train_model import train_model, generate_transformers, generate_kornia_transforms
from tqdm import trange

class WSI_Dataset(Dataset):
    def __init__(self, patches, transform):
        self.patches=patches
        self.to_pil=lambda x: Image.fromarray(x)
        self.length=len(self.patches)
        self.transform=transform
        
    def __getitem__(self,idx):
        X=self.transform(self.to_pil(self.patches[idx]))
        return X,torch.zeros(X.shape[-2:]).unsqueeze(0).long()
    
    def __len__(self):
        return self.length
    
def predict_nuclei(basename="163_A1a",
                   gpu_id=0):
    analysis_type="tumor"
    patch_size=256
    patch_info_file,npy_file=f"patches/{basename}_{analysis_type}_map.pkl",f"patches/{basename}_{analysis_type}_map.npy"
    patches=np.load(npy_file)
    custom_dataset=WSI_Dataset(patches,generate_transformers(256,256)['test'])
    Y_seg=train_model(inputs_dir='inputs',
                    architecture='resnet50',
                    batch_size=512,
                    num_classes=2,
                    predict=True,
                    model_save_loc="models/nuclei.pth",
                    predictions_save_path='tmp_test.pkl',
                    predict_set='custom',
                    verbose=False,
                    class_balance=False,
                    gpu_id=gpu_id,
                    tensor_dataset=False,
                    semantic_segmentation=True,
                    custom_dataset=custom_dataset,
                    save_predictions=False)['pred']

    xy=pd.read_pickle(patch_info_file)[['x','y']].values
    img_shape=np.load(f"inputs/{basename}.npy",mmap_mode="r").shape[:-1]
    pred_mask=np.zeros(img_shape)
    for i in trange(Y_seg.shape[0]):
        x,y=xy[i]
        pred_mask[x:x+patch_size,y:y+patch_size]=Y_seg[i].argmax(0)
    pred_mask=pred_mask.astype(bool)
    np.save(f"nuclei_results/{basename}.npy",pred_mask)


In [33]:
# predict_nuclei(basename="163_A1a",
#                    gpu_id=0)

In [34]:
# estimate Mapper graphs macro+tumor

In [35]:
from torch_cluster import nearest
import sys, os, torch, numpy as np, pandas as pd
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
sys.path.insert(0,os.path.abspath("./dgm/"))
from dgm.dgm import DGM
from umap import UMAP
import pickle

classes_=['dermis', 'epidermis', 'hole', 'subcutaneous tissue']

def relabel_tumor(graph_tumor,graph_macro):
    le=LabelEncoder().fit(classes_)
    re_idx=nearest(torch.tensor(graph_tumor['xy']), torch.tensor(graph_macro['xy'])).numpy()
    unassigned=(graph_tumor['xy']-graph_macro['xy'][re_idx]).sum()!=0
    macro_pred=graph_macro['y_pred'].argmax(1)
    tumor_pred=graph_tumor['y_pred'].argmax(1)
    benign=tumor_pred==0
    tumor_pred=tumor_pred.astype('str')
    tumor_pred[benign]=le.inverse_transform(macro_pred[re_idx][benign])
    tumor_pred[~benign]='tumor'
    tumor_pred[unassigned]='unassigned'
    graph_tumor['annotation']=tumor_pred
    return graph_tumor

def construct_mapper(graph):
    z=UMAP(n_components=2,random_state=42).fit_transform(graph['z'])
    return dict(out_res=DGM(num_intervals=2,overlap=0.01,min_component_size=100,eps=0.1, sdgm=True).fit_transform(graph['G'], z),graph=graph)

def get_interaction(out_graph,y_orig,res,lb=None,plot=False,le=None):
    if not isinstance(lb,type(None)):
        y_orig=lb.transform(y_orig)
    node_makeup={}# only if predict
    for node in out_graph.nodes():
        nodes=res['mnode_to_nodes'][node]
        node_makeup[node]=y_orig[np.array(list(nodes))].mean(0)
    edges = out_graph.edges()
    edge_weight=res['edge_weight']
    weights = np.array([edge_weight[(min(u, v), max(u, v))] for u, v in edges], dtype=np.float32)
    edgelist=list(edges)
    A=np.zeros((len(lb.classes_),len(lb.classes_)))
    for i in range(len(edgelist)):
        send=node_makeup[edgelist[i][0]]
        receive=node_makeup[edgelist[i][1]]
        a=np.outer(send,receive)
        a=(a+a.T)/2.*weights[i]
        A+=a
    invasion_mat=pd.DataFrame(A,columns=le.inverse_transform(np.arange(len(lb.classes_))),index=le.inverse_transform(np.arange(len(lb.classes_))))
    return invasion_mat

def calc_hole_vals(dgm_result,weights={'dermis':3,'epidermis':2,'subcutaneous tissue':1}):
    y_pred=dgm_result['graph']['y_pred'].argmax(1)
    out_graph,res=dgm_result['out_res']
    le=LabelEncoder().fit(classes_)
    area_hole=(le.inverse_transform(y_pred)=='hole').mean()
    hole_share=get_interaction(out_graph,y_pred,res,lb=LabelBinarizer().fit(np.arange(len(le.classes_))),le=le)['hole']
    hole_share=hole_share.loc[hole_share.index!='hole']
    hole_share=pd.DataFrame(hole_share).reset_index()
    hole_share2=hole_share.set_index('index')
    hole_share2['weight']=pd.Series(weights)
    hole_share2['importance']=hole_share2['weight']*hole_share2['hole']
    return hole_share2

def calc_tumor_vals(dgm_result,weights={'dermis':1,'epidermis':1,'subcutaneous tissue':1,'hole':1}):
    out_graph,res=dgm_result['out_res']
    le=LabelEncoder().fit(dgm_result['graph']['annotation'])
    y=le.transform(dgm_result['graph']['annotation'])
    tumor_share=get_interaction(out_graph,y,res,lb=LabelBinarizer().fit(np.arange(len(le.classes_))),le=le)['tumor']
    tumor_share=tumor_share.loc[~tumor_share.index.isin(['tumor','unassigned'])]
    tumor_share=pd.DataFrame(tumor_share).reset_index()
    tumor_share2=tumor_share.set_index('index')
    tumor_share2['weight']=pd.Series(weights)
    tumor_share2['importance']=tumor_share2['weight']*tumor_share2['tumor']
    return tumor_share2

def generate_quality_scores(basename):
    graphs={k:torch.load(os.path.join("gnn_results",f"{basename}_{k}_map.pkl")) for k in ['tumor','macro']}
    graphs['tumor']=[relabel_tumor(graph_tumor,graph_macro) for graph_tumor,graph_macro in zip(graphs['tumor'],graphs['macro'])]

    mapper_graphs=dict()
    for k in ['tumor','macro']:
        mapper_graphs[k]=[construct_mapper(graph) for graph in graphs[k]]

    scoring_fn=dict(tumor=calc_tumor_vals,macro=calc_hole_vals)
    quality_score=dict()

    for k in mapper_graphs:
        quality_score[k]=pd.concat([scoring_fn[k](dgm_result)['importance'] for dgm_result in mapper_graphs[k]],axis=1)
        quality_score[k].columns=[f'section_{i}' for i in range(1,len(quality_score[k].columns)+1)]

    pickle.dump(quality_score,open(f'quality_scores/{basename}.pkl','wb'))

In [36]:
# ink prediction

In [37]:
from skimage import morphology as morph
from scipy.ndimage import binary_opening, binary_dilation, label as scilabel
from skimage import filters, measure
from skimage.morphology import disk
import numpy as np, pandas as pd, copy
import sys,os,cv2
from itertools import product
sys.path.insert(0,os.path.abspath('.'))
from filters import filter_red_pen, filter_blue_pen, filter_green_pen

def filter_yellow(img): # https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/
    img_hsv=cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    return cv2.inRange(img_hsv,(10, 30, 30), (30, 255, 255))

ink_fn=dict(red=filter_red_pen,
           blue=filter_blue_pen,
           green=filter_green_pen,
           yellow=filter_yellow)

ink_min_size=dict(red=100,
           blue=30,
           green=30,
           yellow=1000)

colors=dict(red=np.array([255,0,0]),
           blue=np.array([0,0,255]),
           green=np.array([0,255,0]),
           yellow=np.array([255,255,0]))

def tune_mask(mask,edges,min_size=30):
    mask=(binary_dilation(mask,disk(3,bool),iterations=5) & edges)
    mask=binary_opening(mask,disk(3,bool),iterations=1)
    return morph.remove_small_objects(mask, min_size=min_size, connectivity = 2, in_place=True)>0

def filter_tune(img,color,edges):
    return tune_mask(~ink_fn[color](img),edges,min_size=ink_min_size[color])

def get_edges(mask):
    edges=filters.sobel(mask)>0
    edges = binary_dilation(edges,disk(30,bool))
    return edges

def detect_inks(basename="163_A1a",
                compression=8):
    img,mask=np.load(f"inputs/{basename}.npy"),np.load(f"masks/{basename}_macro_map.npy")
    img=cv2.resize(img,None,fx=1/compression,fy=1/compression)
    mask=cv2.resize(mask.astype(int),None,fx=1/compression,fy=1/compression,interpolation=cv2.INTER_NEAREST).astype(bool)
    labels,n_objects=scilabel(mask)
    edges=get_edges(mask)
    pen_masks={k:filter_tune(img,k,edges) for k in ink_fn}

    for k in ['green','blue','red','yellow']:
        img[pen_masks[k],:]=colors[k]

    coords_df=pd.DataFrame(index=list(ink_fn.keys())+["center_mass"],columns=np.arange(1,n_objects+1))
    for color,obj in product(coords_df.index[:-1],coords_df.columns):
        coords_df.loc[color,obj]=np.vstack(np.where((labels==obj) & (pen_masks[color]))).T*compression
    for obj in coords_df.columns:
        coords_df.loc["center_mass",obj]=np.vstack(np.where(labels==obj)).T.mean(0)*compression

    coords_df.to_pickle(f"detected_inks/{basename}.pkl")
    np.save(f"detected_inks/{basename}_thumbnail.npy",img)

In [58]:
def run_workflow_series(basename):
    print(f"{basename} preprocessing")
    preprocess(basename=basename,
               threshold=0.05,
               patch_size=256)
    
    for k in ['tumor','macro']:
        print(f"{basename} {k} embedding")
        generate_embeddings(basename=basename,
                            analysis_type=k,
                           gpu_id=0)

        print(f"{basename} {k} build graph")
        create_graph_data(basename=basename,
                          analysis_type=k,
                          radius=256,
                          min_component_size=600)
        
        print(f"{basename} {k} gnn predict")
        predict(basename=basename,
                analysis_type=k,
                gpu_id=0)

    print(f"{basename} quality assessment")
    generate_quality_scores(basename)
    
    print(f"{basename} ink detection")
    detect_inks(basename=basename,
                compression=8)
    
    print(f"{basename} nuclei detection")
    predict_nuclei(basename=basename,
                   gpu_id=0)
    

In [61]:
import glob, os
def run_series(patient="163_A1"):
    for f in glob.glob(f"inputs/{patient}*.npy"):
        run_workflow_series(os.path.basename(f).replace(".npy",""))

In [ ]:
# blend predicted scores
# do later

In [ ]:
# organize / cut sections

In [16]:
import glob,pickle, numpy as np

def dump_results(patient="163_A1",scheme="2/1"):
    n_sections,n_blocks_per_section=np.array(scheme.split("/")).astype(int)
    images=sorted(glob.glob(f"inputs/{patient}*.npy"))
    masks=sorted(glob.glob(f"inputs/{patient}*.npy"))
    gnn_results=sorted(glob.glob(f"gnn_results/{patient}*.pkl"))
    quality_scores=sorted(glob.glob(f"quality_scores/{patient}*.pkl"))
    ink_results=sorted(glob.glob(f"detected_inks/{patient}*.pkl"))
    nuclei_results=sorted(glob.glob(f"nuclei_results/{patient}*.npy"))

    pickle.dump(dict(n_sections=n_sections,
                    n_blocks_per_section=n_blocks_per_section,
                    images=images,
                    masks=masks,
                    gnn_results=gnn_results,
                    quality_scores=quality_scores,
                    ink_results=ink_results,
                    nuclei_results=nuclei_results),open(f'results/{patient}.pkl','wb'))


In [17]:
# estimate alignment parameters
# do later

In [18]:
# apply slide level alignment parameters to each exported image + mask
# original, macro, tumor, nuclei
# do later

In [ ]:
# quality score from adjacent sections after extracting mapper scores

In [28]:
import pandas as pd
patient="163_A1"

results=pd.read_pickle(f"results/{patient}.pkl")
quality_score={k:dict(enumerate([pd.concat([pd.read_pickle(f)[k].iloc[:,section::results['n_blocks_per_section']].fillna(0) for f in results['quality_scores']],axis=1) for section in range(results['n_blocks_per_section'])])) for k in ['tumor','macro']}
# score per block per section ADD
# average across blocks?

In [ ]:
# build dzi

In [ ]:
# report generation

In [ ]:
# use dash to call prelim openseadragons; update with fine-tuned heatmaps

In [ ]:
# add more here; turn each cell into python script after testing
# arcticai package
# airflow scripts

In [ ]:
# OLD

def filter_mask(mask): # fill holes here (2 masks output) and ensure only have top X sections; find largest sections; do later
    macro_mask=fill_holes(mask)
    return mask, macro_mask

# holes for certain analysis type; rip from pathflow and lower dependencies
# preprocess
# analysis_type=""
# turn into custom dataset for pathpretrain eval
# predict(hidden_topology=[32]*3)
# predict_nuclei()